In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/02 14:00:27 WARN Utils: Your hostname, codespaces-2912e2 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/03/02 14:00:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/02 14:00:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.registerTempTable('green')

/home/vscode/.local/lib/python3.9/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [4]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [5]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [6]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [7]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [8]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [9]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [10]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [11]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [12]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [13]:
df_join = spark.read.parquet('data/report/revenue/total')

In [14]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [15]:
df_zones = spark.read.parquet('zones/')

In [16]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [17]:
df_result.show(5)

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+--------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID| Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+--------+--------------------+------------+
|2020-01-01 00:00:00|  10|              NULL|                NULL|             42.41|                    2|        10|  Queens|        Baisley Park|   Boro Zone|
|2020-01-01 00:00:00|  14|              NULL|                NULL|               8.8|                    1|        14|Brooklyn|           Bay Ridge|   Boro Zone|
|2020-01-01 00:00:00|  15|              NULL|                NULL|             34.09|                    1|        15|  Queens|Bay Terrace/Fort ...|   Boro Zone|
|2020-01-01 00:00:00|  17|19

In [18]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')